# Constants

In [36]:
import datetime
import math

In [37]:
s_to_hr = 1/(60*60)

defect_rate_Li = 1/4000     # [defect/item]
defect_rate_Al = 1/2000     # [defect/item]
cells_per_pack_Al = 40      # [cells]
cells_per_pack_Li = 30      # [cells]

uptime_day_ratio = 20/24

component_failure_probibility = 0.01
component_failure_downtime = 0.5        # [Hr]

conveyor_cost_per_hr = 10   # [£/Hr]
conveyor_repair_cost = 500  # [£/repair]
conveyor_upfront = 10000    # [£]
conveyor_speed = 0.1        # [Hr]

arm_cost_per_hr = 10        # [£/Hr]
arm_repair_cost = 500       # [£/repair]
arm_upfront = 125000        # [£]

enclosure_removal = 15*s_to_hr  # [s]

# Existing Layout

In [38]:
# Initialise quantities of components and repairs
num_components = 4

effective_num_conveyor = 3
num_conveyor = 3
effective_num_arms = 1
num_arms = 1

arm_repairs = 0
conveyor_repairs = 0

# Ideal cycle time
ideal_cycle_time = (9)*s_to_hr
adjusted_cycle_time_first = ideal_cycle_time

# Initialise calculation quantities
avg_cells_per_pack = cells_per_pack_Li
stage_one_cells = 10000*cells_per_pack_Li
stage_one_packs = stage_one_cells/avg_cells_per_pack
average_defect_rate = defect_rate_Li

# Adjust for cell failures
adjusted_cycle_time_first += ideal_cycle_time*average_defect_rate

# Adjust for line stiops
arm_repairs += effective_num_arms*component_failure_probibility*stage_one_packs
conveyor_repairs += effective_num_conveyor*component_failure_probibility*stage_one_packs
adjusted_cycle_time_first += component_failure_downtime*num_components*component_failure_probibility*(stage_one_packs/stage_one_cells)

# Adjust for pack removal (time is halved as one line is removing packs, the other can continue)
adjusted_cycle_time_first += enclosure_removal*(stage_one_packs/stage_one_cells)

# Adjust for uptime
adjusted_cycle_time_first /= uptime_day_ratio

# Final calculations
batch_produciton_time = 10000*cells_per_pack_Li*adjusted_cycle_time_first
production_efficiency = (ideal_cycle_time/adjusted_cycle_time_first)*100

# Calculate costs
line_cost = conveyor_upfront*num_conveyor + arm_upfront*num_arms
line_cost_per_hour = arm_cost_per_hr*num_arms + conveyor_cost_per_hr*num_conveyor
repair_cost_per_hour = (conveyor_repairs*conveyor_repair_cost + arm_repairs*arm_repair_cost)/batch_produciton_time

# Display results
print("Production line upfront cost: £%.2f" %(line_cost))
print("Production line cost per hour £%.2f, totaling £%.2f across produciton cycle" %(line_cost_per_hour, line_cost_per_hour*batch_produciton_time))
print("Total repair cost per hour £%.2f, totaling £%.2f across produciton cycle" %(repair_cost_per_hour, repair_cost_per_hour*batch_produciton_time))
print("Total production cost £%.2f" %(line_cost + line_cost_per_hour*batch_produciton_time + repair_cost_per_hour*batch_produciton_time))


print("Production rate of %.2f cells/hr" %(1/adjusted_cycle_time_first))
print("Production efficiency of %.2f%%" %(production_efficiency))
print("Batch produciton time %.2f days or %.2f hrs" %(batch_produciton_time/24, batch_produciton_time))
operational_day = datetime.date(2023, 1, 1) - datetime.timedelta(days=math.ceil(batch_produciton_time/24))
formatted_date = operational_day.strftime("%B-%d")
print("Earliest operational day is", formatted_date)

Production line upfront cost: £155000.00
Production line cost per hour £40.00, totaling £47609.00 across produciton cycle
Total repair cost per hour £168.04, totaling £200000.00 across produciton cycle
Total production cost £402609.00
Production rate of 252.05 cells/hr
Production efficiency of 63.01%
Batch produciton time 49.59 days or 1190.23 hrs
Earliest operational day is November-12


# Option 1

In [39]:
num_components_tot = 8
num_arms_tot = 2
num_conveyor_tot = 6

# Initialise repair counters
arm_repairs = 0
conveyor_repairs = 0
stage_running_cost = 0

########################### STAGE 1 #################################
# Initialise effective component counts
effective_num_components = 4
effective_num_conveyor = 3
num_conveyor = 6
num_arms = 2

# Ideal cycle time
ideal_cycle_time = (9/2)*s_to_hr
adjusted_cycle_time_first = ideal_cycle_time

# Calculate cells, packs and avg defect rate for stage 1
stage_one_cells = 5000*2*cells_per_pack_Li
stage_one_packs = (stage_one_cells/2)*(1/cells_per_pack_Li + 1/cells_per_pack_Al)
average_defect_rate = (defect_rate_Li + defect_rate_Al)/2

# Adjust for cell failures
adjusted_cycle_time_first += ideal_cycle_time*average_defect_rate

# Adjust for line stiops
arm_repairs += component_failure_probibility*stage_one_packs
conveyor_repairs += effective_num_conveyor*component_failure_probibility*stage_one_packs
adjusted_cycle_time_first += component_failure_downtime*effective_num_components*component_failure_probibility*(stage_one_packs/stage_one_cells)

# Adjust for pack removal (time is halved as one line is removing packs, the other can continue)
adjusted_cycle_time_first += enclosure_removal*(stage_one_packs/stage_one_cells)

# Adjust for uptime
adjusted_cycle_time_first /= uptime_day_ratio

# Calculate repari costs
stage_running_cost += adjusted_cycle_time_first*stage_one_cells*(conveyor_cost_per_hr*num_conveyor + arm_cost_per_hr*num_arms)

########################### STAGE 2 #################################
# Calculate average arm/conveyor daily uptime
effective_num_components = 4
effective_num_conveyor = 3
num_conveyor = 3
num_arms = 1

# Ideal cycle time
ideal_cycle_time = (9/2)*s_to_hr
adjusted_cycle_time_second = ideal_cycle_time*2

# Calculate cells, packs and avg defect rate for stage 2
stage_two_cells = 5000*(cells_per_pack_Al - cells_per_pack_Li)
stage_two_packs = stage_two_cells/cells_per_pack_Al
average_defect_rate = defect_rate_Al

# Adjust for cell failures
adjusted_cycle_time_second += ideal_cycle_time*average_defect_rate

# Adjust for line stiops
arm_repairs += component_failure_probibility*stage_one_packs
conveyor_repairs += effective_num_conveyor*component_failure_probibility*stage_one_packs
adjusted_cycle_time_second += component_failure_downtime*effective_num_components*component_failure_probibility*(stage_two_packs/stage_two_cells)

# Adjust for pack removal (time is halved as one line is removing packs, the other can continue)
adjusted_cycle_time_second += enclosure_removal*(stage_two_packs/stage_two_cells)

# Adjust for uptime
adjusted_cycle_time_second /= uptime_day_ratio

# Calculate repari costs
stage_running_cost += adjusted_cycle_time_second*stage_two_cells*(conveyor_cost_per_hr*num_conveyor + arm_cost_per_hr*num_arms)

########################### CALCS #################################
batch_produciton_time = adjusted_cycle_time_first*stage_one_cells + adjusted_cycle_time_second*stage_two_cells
adjusted_cycle_time = batch_produciton_time/(stage_one_cells + stage_two_cells)
production_efficiency = (ideal_cycle_time/adjusted_cycle_time)*100

# Calculate costs
line_cost = conveyor_upfront*num_conveyor_tot + arm_upfront*num_arms_tot
line_cost_per_hour = stage_running_cost/batch_produciton_time
repair_cost_per_hour = (conveyor_repairs*conveyor_repair_cost + arm_repairs*arm_repair_cost)/batch_produciton_time

# Display results
print("Production line upfront cost: £%.2f" %(line_cost))
print("Production line cost per hour £%.2f, totaling £%.2f across produciton cycle" %(line_cost_per_hour, line_cost_per_hour*batch_produciton_time))
print("Total repair cost per hour £%.2f, totaling £%.2f across produciton cycle" %(repair_cost_per_hour, repair_cost_per_hour*batch_produciton_time))
print("Total line cost is £%.2f" %(line_cost + line_cost_per_hour*batch_produciton_time + repair_cost_per_hour*batch_produciton_time))


print("Production rate of %.2f cells/hr" %(1/adjusted_cycle_time))
print("Production efficiency of %.2f%%" %(production_efficiency))
print("Batch produciton time %.2f days or %.2f hrs" %(batch_produciton_time/24, batch_produciton_time))
operational_day = datetime.date(2023, 1, 1) - datetime.timedelta(days=math.ceil(batch_produciton_time/24))
formatted_date = operational_day.strftime("%B-%d")
print("Earliest operational day is", formatted_date)

Production line upfront cost: £310000.00
Production line cost per hour £71.63, totaling £63765.00 across produciton cycle
Total repair cost per hour £393.17, totaling £350000.00 across produciton cycle
Total line cost is £723765.00
Production rate of 393.17 cells/hr
Production efficiency of 49.15%
Batch produciton time 37.09 days or 890.21 hrs
Earliest operational day is November-24


# Option 2

In [40]:
num_components_tot = 5
num_arms_tot = 2
num_conveyor_tot = 3

# Initialise repair counters
arm_repairs = 0
conveyor_repairs = 0
stage_running_cost = 0

########################### STAGE 1 #################################
# Calculate average arm/conveyor daily uptime
effective_num_components = 4
effective_num_conveyor = 3
num_conveyor = 3
num_arms = 2

# Ideal cycle time
ideal_cycle_time = (10)*s_to_hr
adjusted_cycle_time_first = ideal_cycle_time

# Calculate cells, packs and avg defect rate for stage 1
stage_one_cells = 5000*2*cells_per_pack_Li
stage_one_packs = (stage_one_cells/2)*(1/cells_per_pack_Li + 1/cells_per_pack_Al)
average_defect_rate = (defect_rate_Li + defect_rate_Al)/2

# Adjust for cell failures
adjusted_cycle_time_first += ideal_cycle_time*average_defect_rate

# Adjust for line stiops
arm_repairs += component_failure_probibility*stage_one_packs
conveyor_repairs += effective_num_conveyor*component_failure_probibility*stage_one_packs
adjusted_cycle_time_first += component_failure_downtime*effective_num_components*component_failure_probibility*(stage_one_packs/stage_one_cells)

# Adjust for pack removal (time is halved as one line is removing packs, the other can continue)
adjusted_cycle_time_first += enclosure_removal*(stage_one_packs/stage_one_cells)

# Adjust for uptime
adjusted_cycle_time_first /= uptime_day_ratio

# Calculate repari costs
stage_running_cost += adjusted_cycle_time_first*stage_one_cells*(conveyor_cost_per_hr*num_conveyor + arm_cost_per_hr*num_arms)

########################### STAGE 2 #################################
# Calculate average arm/conveyor daily uptime
effective_num_components = 4
effective_num_conveyor = 3
num_conveyor = 3
num_arms = 1

# Ideal cycle time
ideal_cycle_time = (9)*s_to_hr
adjusted_cycle_time_second = ideal_cycle_time

# Calculate cells, packs and avg defect rate for stage 2
stage_two_cells = 5000*(cells_per_pack_Al - cells_per_pack_Li)
stage_two_packs = stage_two_cells/cells_per_pack_Al
average_defect_rate = defect_rate_Al

# Adjust for cell failures
adjusted_cycle_time_second += ideal_cycle_time*average_defect_rate

# Adjust for line stiops
arm_repairs += component_failure_probibility*stage_one_packs
conveyor_repairs += effective_num_conveyor*component_failure_probibility*stage_one_packs
adjusted_cycle_time_second += component_failure_downtime*effective_num_components*component_failure_probibility*(stage_two_packs/stage_two_cells)

# Adjust for pack removal (time is halved as one line is removing packs, the other can continue)
adjusted_cycle_time_second += enclosure_removal*(stage_two_packs/stage_two_cells)

# Adjust for uptime
adjusted_cycle_time_second /= uptime_day_ratio

# Calculate repari costs
stage_running_cost += adjusted_cycle_time_second*stage_two_cells*(conveyor_cost_per_hr*num_conveyor + arm_cost_per_hr*num_arms)

########################### CALCS #################################
batch_produciton_time = adjusted_cycle_time_first*stage_one_cells + adjusted_cycle_time_second*stage_two_cells
adjusted_cycle_time = batch_produciton_time/(stage_one_cells + stage_two_cells)

production_efficiency = (ideal_cycle_time/adjusted_cycle_time)*100

line_cost = conveyor_upfront*num_conveyor_tot + arm_upfront*num_arms_tot
line_cost_per_hour = stage_running_cost/batch_produciton_time
repair_cost_per_hour = (conveyor_repairs*conveyor_repair_cost + arm_repairs*arm_repair_cost)/batch_produciton_time

print("Production line upfront cost: £%.2f" %(line_cost))
print("Production line cost per hour £%.2f, totaling £%.2f across produciton cycle" %(line_cost_per_hour, line_cost_per_hour*batch_produciton_time))
print("Total repair cost per hour £%.2f, totaling £%.2f across produciton cycle" %(repair_cost_per_hour, repair_cost_per_hour*batch_produciton_time))
print("Total line cost is £%.2f" %(line_cost + line_cost_per_hour*batch_produciton_time + repair_cost_per_hour*batch_produciton_time))

print("Production rate of %.2f cells/hr" %(1/adjusted_cycle_time))
print("Production efficiency of %.2f%%" %(production_efficiency))
print("Batch produciton time %.2f days or %.2f hrs" %(batch_produciton_time/24, batch_produciton_time))
operational_day = datetime.date(2023, 1, 1) - datetime.timedelta(days=math.ceil(batch_produciton_time/24))
formatted_date = operational_day.strftime("%B-%d")
print("Earliest operational day is", formatted_date)

Production line upfront cost: £280000.00
Production line cost per hour £48.71, totaling £70159.25 across produciton cycle
Total repair cost per hour £242.98, totaling £350000.00 across produciton cycle
Total line cost is £700159.25
Production rate of 242.98 cells/hr
Production efficiency of 60.74%
Batch produciton time 60.02 days or 1440.45 hrs
Earliest operational day is November-01
